# Movie Tool

In [ ]:
%pip install requests
%pip install pandas
%pip install beautifulsoup4
%pip install cinemagoer

In [2]:
import pandas as pd
import numpy as np
import time
import datetime
import utils

# IMDB_API_KEY = 'k_e9n79f4d' # daibanglam
IMDB_API_KEY = 'k_x9atwnz9' # gravitylabs

# the cookie for making API request to Netflix
# it can be retrieved from Chrome Dev Tool, after logging to Netflix
cookie = 'memclid=b783e450-fa77-488c-b3e2-beff624d26da; nfvdid=BQFmAAEBEEVEA6OC5bEMUUnOPrODS8pgtswkpg-fPl9nVBvzgBL6h4xcvm1AW25uw7kCk3ontTzlyk_IFIznVD7tQB97u8Sb9yS3ZTnKWCiKr7ML5MnwT0FDrZ5vU5D__d-e0TaPHF35IjWRoAUBT5CD29YZRJBC; netflix-sans-normal-3-loaded=true; netflix-sans-bold-3-loaded=true; SecureNetflixId=v%3D2%26mac%3DAQEAEQABABSAzFtR1_DQDeTeE_cs6BSVZTnOPPH7nhw.%26dt%3D1678850528044; NetflixId=ct%3DBQAOAAEBEPZtIjiITaLAjPHmT8jTaq6CUJGFXdchN_Zw-Vx7cAp_OLQ4n1G40FIEEK7V31GV7ekZfVgrcYKqn5MiwcSe29CMiHtsofe2gCh82PpPrA73UhReC-q_dzk5uevCmNNqO0WSa9kZ43iv1s3gOBSLRVVb-DGNKlwycF5hjDc35qDaHwu_CFr3_IGHxJv3penntkSncxthYn-7nPjP55hxpY3igY92-RliOVFx_AByre7Y67bbFX_HqKUMHM9korEcHEqNFGmvMwzynjNidRg4AcR_amRe6L76e9EwFQOObonBiBg5GUe4N0nNZtk5UPCW66cbQaUqdiKQLZuym2IXqeddHQXJEnhia4E-8yV-b67xHWt954Whefxdup61cVwvNsByhcbNXi3NGrbNJYkJtCVfd082MLjjTEFtvWAXUa3q9k17iab3NEE_awvaqhjeHSI2ZllJkH0GC57LZ4S5ICXLXnmpvNbjFjaDXAWheR7rFViVBTg-MlJZMRtmaWfa8N-Ho6_DP7a6_vqjtz6LYkVAtrju_h66QGuUDmfSbXGsfF3T9OcDW1ei2mvKIh6HQG2LzGvGR8yFSUXLKvth5C3HTph9GnxNVUW0cad136VQALpw81bxbdH_bd3xjL0EXSRNAJMJSywgllhFefBzEjCwV42maxqdVFi6AUDi1uJPw4Y6mLn4P-AwXxveScx8feUM9qSy5jlZUMTdVVwgsKNarvLVzgaUZF0q9NrkmQPvP1ZQNE20c8eOBgOuJYA8_Yx8zC0PKvvcXC1tKHrP9Ep16LqNbJte6cIAjWRDuV7nFgY.%26bt%3Ddbl%26ch%3DAQEAEAABABQnw__eUldKi0gl_1tUcwDpV3QssXhziJU.%26v%3D2%26mac%3DAQEAEAABABRdvju3TEOdz_bObCmuQ6s-5JmLA592VCg.; profilesNewSession=0; OptanonConsent=isIABGlobal=false&datestamp=Wed+Mar+15+2023+11%3A00%3A35+GMT%2B0700+(Indochina+Time)&version=202301.1.0&consentId=22b81ae4-c299-4a0c-ac36-055cb8f2e01c&interactionCount=1&landingPath=NotLandingPage&groups=C0001%3A1%2CC0002%3A1%2CC0003%3A1%2CC0004%3A1&hosts=&AwaitingReconsent=false&isGpcEnabled=0'

imdb_client = utils.CinemagoerImdbStrategy()
netflix_client = utils.HttpNetflixStrategy(cookie)

## Load movie list

In [3]:
mov_file = 'data/movies.csv'
df = pd.read_csv(mov_file, index_col='Netflix Id')
len(df)

565

## Save movie list

In [ ]:
df.to_csv('data/movies.csv')

## Get My List movies

Fetch movies from My List

In [ ]:
my_list_movies = netflix_client.get_my_list()
len(my_list_movies)

Add new movies to movie list

In [ ]:
new_movies = [mov for mov in my_list_movies if mov['Netflix Id'] not in df.index]
if len(new_movies) > 0:
  df = pd.concat([df, pd.DataFrame.from_records(new_movies, index='Netflix Id')])
len(df)

## Get Last day to watch on Netflix

In [ ]:
now = datetime.date.today()
for index, row in df.iterrows():
  # skip it has been processed
  last_fetch = df.at[index, 'Last Fetch']
  if isinstance(last_fetch, str) and (now - datetime.datetime.strptime(last_fetch, '%Y-%m-%d').date()).days < 5:
    continue
  
  # skip if is Netflix original
  if df.at[index, 'Netflix Original']:
    continue

  df.at[index, 'Last Day To Watch'] = netflix_client.get_last_date_to_watch(index)
  df.at[index, 'Last Fetch'] = now.isoformat()

## Get genres and rating

In [ ]:
df2 = df.sort_values(by='Last Day To Watch', ascending=True, na_position='last')
df2.head()
now = datetime.date.today()

for index, row in df2.iterrows():
  if not np.isnan(row['Imdb Rating']):
    continue
  
  print(f"processing movie {row['Title']}")
  info = imdb_client.get_movie_info(row['Title'], row['Release Year'])
  if info is None:
    continue
  
  df.at[index, 'Genres'] = ','.join(info.genres)
  df.at[index, 'Imdb Rating'] = info.rating
  time.sleep(1)

## Find movies to watch

In [8]:
from IPython.display import display, HTML

df2 = df.copy()

# filter movies by genres
# df2 = df2[df2['Genres'].str.contains('Comedy')]

# priority: expiring movies, single movies, not from Netflix, highest rating
df2['Movie Type Rank'] = pd.Categorical(df2['Movie Type'], categories=['Movie', 'TV Series'], ordered=True)
df2.sort_values(
  by=['Last Day To Watch', 'Movie Type Rank', 'Netflix Original', 'Imdb Rating'], 
  ascending=[True, True, True, False], 
  na_position='last',
  inplace=True)

# show movie list
df2['Netflix Link'] = df2.index
display(HTML(df2.to_html(
  columns=['Image', 'Title', 'Last Day To Watch', 'Imdb Rating', 'Genres', 'Release Year', 'Movie Type', 'Netflix Link'],
  formatters={
    'Netflix Link': lambda val: f'<a href="https://www.netflix.com/title/{val}">{val}</a>',
    'Image': lambda val: f'<img src="{val}" width="171" height="96" />',
  },
  escape=False,
  index=False, 
)))

Image,Title,Last Day To Watch,Imdb Rating,Genres,Release Year,Movie Type,Netflix Link
,Oblivion,2023-03-17,7.0,"Action,Adventure,Sci-Fi",2013,Movie,70259170
,Darwin’s Game,2023-03-19,7.2,"Animation,Action,Drama,Mystery",2020,TV Series,81234353
,Legend of Yun Xi,2023-03-25,7.3,"Fantasy,History,Romance",2018,TV Series,81293269
,Now You See Me,2023-03-26,7.2,"Crime,Mystery,Thriller",2013,Movie,70243464
,Now You See Me 2,2023-03-26,6.4,"Action,Adventure,Comedy",2016,Movie,80093138
,Resident Evil: Vendetta,2023-03-30,6.2,"Animation,Action,Adventure",2017,Movie,80190911
,The Cable Guy,2023-03-30,6.1,"Comedy,Drama,Thriller",1996,Movie,342807
,The Dark Knight Rises,2023-03-31,8.4,"Action,Drama",2012,Movie,70213514
,Batman Begins,2023-03-31,8.2,"Action,Crime,Drama",2005,Movie,70021642
,How to Train Your Dragon,2023-03-31,8.1,"Animation,Action,Adventure",2010,Movie,70109893
